<a href="https://colab.research.google.com/github/flake313/Malashin/blob/main/9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import random
import math

class MonteCarloMassServiceSystem:
    def __init__(self):
        self.servers = [True, True]  # Серверы: свободен или занят
        self.queue = []  # Очередь клиентов
        self.customer_count = 0  # Общее количество клиентов
        self.total_service_time = 0  # Общее время обслуживания
        self.total_queue_wait_time = 0  # Общее время ожидания в очереди
        self.total_system_wait_time = 0  # Общее время ожидания в системе
        self.total_queue_length = 0  # Общая длина очереди
        self.simulation_time = 1000  # Время симуляции
        self.current_time = 0  # Внутренние часы симуляции
        self.server_utilization = [0, 0]  # Загрузка серверов

    def simulate(self, lambda_, mu):
        """ Основной метод симуляции системы массового обслуживания """
        while self.current_time < self.simulation_time:
            self.current_time += 1

            # Поступление новых клиентов
            self.generate_customers(lambda_)

            # Обработка клиентов серверами
            self.process_servers(mu)

            # Проверка на занятость всех серверов и увеличение времени в системе
            if self.queue and not any(self.servers):
                self.total_system_wait_time += self.current_time - self.queue[0]

            # Освобождение серверов
            self.release_servers()

        # Печать результатов
        self.print_results(lambda_, mu)

    def generate_customers(self, arrival_rate):
        """ Генерация новых клиентов и добавление их в очередь """
        for _ in range(arrival_rate):
            self.queue.append(self.current_time)  # Клиент поступает в очередь
            self.customer_count += 1  # Увеличиваем счетчик клиентов

    def process_servers(self, service_rate):
        """ Обработка клиентов серверами, если они свободны """
        for i in range(2):
            if self.servers[i] and self.queue:
                customer_arrival_time = self.queue.pop(0)  # Извлечение клиента из очереди
                service_time = 1 / service_rate  # Время обслуживания клиента
                self.total_service_time += service_time

                # Обработка времени ожидания клиента в очереди и системе
                if self.queue:
                    self.total_queue_wait_time += self.current_time - self.queue[0]
                    self.total_system_wait_time += self.current_time - self.queue[0]

                self.servers[i] = False  # Занять сервер
                self.server_utilization[i] += service_time + (self.current_time - customer_arrival_time)

                # Если клиент ожидает долго, увеличиваем общее время ожидания в системе
                if random.random() < 0.6:
                    self.total_system_wait_time += self.current_time - customer_arrival_time

    def release_servers(self):
        """ Освобождение серверов с вероятностью """
        for i in range(2):
            if not self.servers[i]:
                if random.random() < 0.2:  # Вероятность освобождения сервера
                    self.servers[i] = True  # Освобождаем сервер
                    self.total_queue_length += len(self.queue)  # Учитываем длину очереди

    def calculate_probabilities(self, lambda_, mu):
        """ Вычисление вероятностей состояний системы с учетом и λ, и μ """
        # Коэффициент загрузки одного сервера
        rho = lambda_ / (2 * mu)

        # Вероятность, что в системе 0 клиентов (оба сервера свободны)
        P0 = 1 / (1 + (lambda_ / mu) + (lambda_ ** 2) / (2 * mu ** 2))

        # Вероятность, что в системе 1 клиент (один сервер свободен)
        P1 = (lambda_ / mu) / (1 + (lambda_ / mu) + (lambda_ ** 2) / (2 * mu ** 2))

        # Вероятность, что в системе 2 или более клиентов (оба сервера заняты)
        P_busy = 1 - (P0 + P1)

        return P0, P1, P_busy

    def print_results(self, lambda_, mu):
        """ Печать результатов симуляции """
        avg_queue_wait_time = self.total_queue_wait_time / self.customer_count if self.customer_count > 0 else 0
        avg_system_wait_time = self.total_system_wait_time / self.customer_count if self.customer_count > 0 else 0
        avg_queue_length = self.total_queue_length / self.current_time if self.current_time > 0 else 0
        system_utilization = sum(self.server_utilization) / (self.simulation_time * 2)

        # Вывод средних значений
        print(f"Среднее время ожидания в очереди: {avg_queue_wait_time:.2f}")
        print(f"Среднее время ожидания в системе: {avg_system_wait_time:.2f}")
        print(f"Средняя длина очереди: {avg_queue_length:.2f}")
        print(f"Загрузка системы: {system_utilization:.2f}")

        # Вычисление и вывод вероятностей
        P0, P1, P_busy = self.calculate_probabilities(lambda_, mu)
        print(f"Вероятность, что оба сервера заняты: {P_busy:.2f}")
        print(f"Вероятность, что один сервер свободен: {P1:.2f}")
        print(f"Вероятность, что оба сервера свободны: {P0:.2f}")

# Запуск симуляции
lambda_ = 5  # Интенсивность поступления клиентов (λ)
mu = 3  # Интенсивность обслуживания (μ)
system = MonteCarloMassServiceSystem()
system.simulate(lambda_, mu)

Среднее время ожидания в очереди: 37.98
Среднее время ожидания в системе: 153.55
Средняя длина очереди: 953.67
Загрузка системы: 95.05
Вероятность, что оба сервера заняты: 0.34
Вероятность, что один сервер свободен: 0.41
Вероятность, что оба сервера свободны: 0.25


In [39]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon

# Функция для создания звезды
def create_star(radius, num_points, center):
    theta = np.linspace(0, 2 * np.pi, num_points * 2, endpoint=False)
    points = []
    for i, angle in enumerate(theta):
        r = radius if i % 2 == 0 else radius / 2
        points.append((center[0] + r * np.cos(angle), center[1] + r * np.sin(angle)))
    return np.array(points)

# Функция для определения, находится ли точка внутри звезды
def is_inside_star(x, y, star_polygon):
    return star_polygon.contains_point((x, y))

# Функция активации (сигмоида)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Производная сигмоиды
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Прямое распространение
def forward(X, weights_input_hidden, weights_hidden_output):
    print("Forward pass - Input Shape:", X.shape)  # Отладочная информация
    hidden_input = np.dot(X, weights_input_hidden)
    hidden_output = sigmoid(hidden_input)
    final_input = np.dot(hidden_output, weights_hidden_output)
    final_output = sigmoid(final_input)
    return hidden_output, final_output

# Обратное распространение (обучение)
def backprop(X, y, hidden_output, final_output, weights_input_hidden, weights_hidden_output, learning_rate):
    error = y - final_output
    d_output = error * sigmoid_derivative(final_output)

    error_hidden = d_output.dot(weights_hidden_output.T)
    d_hidden = error_hidden * sigmoid_derivative(hidden_output)

    weights_hidden_output += hidden_output.T.dot(d_output) * learning_rate
    weights_input_hidden += X.T.dot(d_hidden) * learning_rate

    return weights_input_hidden, weights_hidden_output

# Параметры сети
input_size = 2  # входные координаты: x и y
hidden_size = 5  # скрытые нейроны
output_size = 1  # результат 0 или 1
learning_rate = 0.1
epochs = 1000

# Инициализация весов
weights_input_hidden = np.random.rand(input_size, hidden_size) - 0.5
weights_hidden_output = np.random.rand(hidden_size, output_size) - 0.5

# Создание звезды
star_radius = 1
num_points = 5
center = (0, 0)
star_points = create_star(star_radius, num_points, center)
star_polygon = Polygon(star_points, closed=True)

# Генерация обучающей выборки
# Создаем точки внутри звезды
num_inside_points = 500
x_inside = np.random.uniform(-1, 1, num_inside_points)
y_inside = np.random.uniform(-1, 1, num_inside_points)

# Фильтруем точки, чтобы они были внутри звезды
X_inside = []
y_inside = []
for x, y in zip(x_inside, y_inside):
    if is_inside_star(x, y, star_polygon):
        X_inside.append([x, y])
        y_inside.append(1)  # внутри звезды

# Создаем точки снаружи звезды
num_outside_points = 500
x_outside = np.random.uniform(-1.5, 1.5, num_outside_points)
y_outside = np.random.uniform(-1.5, 1.5, num_outside_points)

X_outside = []
y_outside = []
for x, y in zip(x_outside, y_outside):
    if not is_inside_star(x, y, star_polygon):
        X_outside.append([x, y])
        y_outside.append(0)  # снаружи звезды

# Объединяем данные
X_train = np.array(X_inside + X_outside)
y_train = np.array(y_inside + y_outside).reshape(-1, 1)

# Проверка форм
print("X_train shape:", X_train.shape)  # Ожидается (1000, 2)
print("y_train shape:", y_train.shape)  # Ожидается (1000, 1)

# Обучение сети
for epoch in range(epochs):
    hidden_output, final_output = forward(X_train, weights_input_hidden, weights_hidden_output)
    weights_input_hidden, weights_hidden_output = backprop(X_train, y_train, hidden_output, final_output, weights_input_hidden, weights_hidden_output, learning_rate)

    if epoch % 100 == 0:
        loss = np.mean(np.square(y_train - final_output))
        print(f'Epoch {epoch}, Loss: {loss}')

# Тестирование и визуализация на случайных точках
num_test_points = 1000
X_test = np.random.uniform(-1.5, 1.5, (num_test_points, 2))

hidden_output, final_output = forward(X_test, weights_input_hidden, weights_hidden_output)
predictions = (final_output > 0.5).astype(int)

# Разделение точек по предсказанию
inside_points = X_test[predictions.flatten() == 1]
outside_points = X_test[predictions.flatten() == 0]

# Визуализация
plt.figure(figsize=(6, 6))
plt.gca().add_patch(Polygon(star_points, closed=True, fill=None, edgecolor='black'))

plt.scatter(inside_points[:, 0], inside_points[:, 1], color='black', label='Inside Star', s=10)
plt.scatter(outside_points[:, 0], outside_points[:, 1], color='red', label='Outside Star', s=10)

plt.xlim([-1.5, 1.5])
plt.ylim([-1.5, 1.5])
plt.gca().set_aspect('equal', adjustable='box')

plt.legend()
plt.show()


X_train shape: (0,)
y_train shape: (0, 1)
Forward pass - Input Shape: (0,)


ValueError: shapes (0,) and (2,5) not aligned: 0 (dim 0) != 2 (dim 0)